In [1]:
import cv2
import numpy as np
import open3d as o3d
import matplotlib.pyplot as plt

from pathlib import Path

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
import utils
import calibration_utils

import importlib
importlib.reload(utils)
importlib.reload(calibration_utils)

<module 'calibration_utils' from '/home/tom/Documents/UNT/csce6260/projects/pcdet_notebooks/calibration_utils.py'>

In [3]:
base = Path.home() / "kitti"
train_dir = base / "training"
velo_dir = train_dir / "velodyne"
calib_dir = train_dir / "calib"
label_dir = train_dir / "label_2"
img_dir = train_dir/ "image_2"

In [4]:
def format_filename(sample_id, ext="txt"):
    return f"{sample_id:06d}.{ext}"

sample_id = 0
bin_path = velo_dir / format_filename(sample_id, "bin")
points = np.fromfile(bin_path, dtype=np.float32).reshape(-1, 4)

In [5]:
def get_min_and_max_values(points):
    min_x, max_x = np.min(points[:, 0]), np.max(points[:, 0])
    min_y, max_y = np.min(points[:, 1]), np.max(points[:, 1])
    min_z, max_z = np.min(points[:, 2]), np.max(points[:, 2])
    return min_x, max_x, min_y, max_y, min_z, max_z

print("Points shape:", points.shape)
print('Min/Max values for all dimensions')
min_x, max_x, min_y, max_y, min_z, max_z = get_min_and_max_values(points)
print(f"X: ({min_x:.2f}, {max_x:.2f})")
print(f"Y: ({min_y:.2f}, {max_y:.2f})")
print(f"Z: ({min_z:.2f}, {max_z:.2f})")

Points shape: (115384, 4)
Min/Max values for all dimensions
X: (-71.04, 73.04)
Y: (-21.10, 53.80)
Z: (-5.16, 2.67)


In [6]:
CLASS_COLORS = {
    1: (1, 0, 0),    # Car     (red)
    2: (0, 1, 0),    # Pedestrian (green)
    3: (0, 0, 1)     # Cyclist (blue)
}

def get_point_colors(z):
    z_norm = (z - z.min()) / (z.max() - z.min() + 1e-6)
    colors = plt.cm.jet(z_norm)[:, :3]
    return colors

def create_point_cloud(points):
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points)
    colors = get_point_colors(points[:, 2])
    pcd.colors = o3d.utility.Vector3dVector(colors)
    return pcd

points = points[:, :3]
pcd = create_point_cloud(points)

In [7]:
def get_cam_to_velo_transform(Tr_velo_to_cam):
    Tr_velo_to_cam = np.vstack((Tr_velo_to_cam, [0, 0, 0, 1]))
    Tr_cam_to_velo = calibration_utils.inverse_rigid_transform(Tr_velo_to_cam)
    return Tr_cam_to_velo

def get_gt_oriented_bounding_boxes(labels, Tr_cam_to_velo):
    gt_oriented_boxes = []
    for label in labels:
        corners_3d_velo = calibration_utils.compute_box_3d(label, Tr_cam_to_velo)
        pcd = o3d.geometry.PointCloud()
        pcd.points = o3d.utility.Vector3dVector(corners_3d_velo)
        oriented_bbox = pcd.get_oriented_bounding_box()
        oriented_bbox.color = (0, 1, 0)
        gt_oriented_boxes.append(oriented_bbox)
    return gt_oriented_boxes


label_file_path = label_dir / format_filename(sample_id, "txt")
labels = utils.parse_label_file(label_file_path)

calib_file_path = calib_dir / format_filename(sample_id, "txt")
calib = utils.parse_calib_file(calib_file_path)
Tr_velo_to_cam = calib['Tr_velo_to_cam']
Tr_cam_to_velo = get_cam_to_velo_transform(Tr_velo_to_cam)

gt_boxes = get_gt_oriented_bounding_boxes(labels, Tr_cam_to_velo)

o3d.visualization.draw_geometries([pcd, *gt_boxes])

### Data Processing (mask/sample/shuffle)

#### Field-of-View (FOV) Filtering

In [8]:
def get_finite_mask(points_img_coords):
    mask_finite = np.isfinite(points_img_coords).all(axis=1)
    return mask_finite

def get_img_mask(points_img_coords, img_shape):
    u = points_img_coords[:, 0]
    v = points_img_coords[:, 1]
    img_height, img_width = img_shape
    mask_img = (u >= 0) & (u < img_width) & (v >= 0) & (v < img_height)
    return mask_img

def get_depth_mask(depth):
    return depth >= 0
    
def get_fov_mask(points_img_coords, img_shape):
    mask_finite = get_finite_mask(points_img_coords)
    mask_img = get_img_mask(points_img_coords, img_shape)
    mask_depth = get_depth_mask(points_img_coords[:, 2])    
    return mask_finite & mask_img & mask_depth

def get_image_shape(img_path):
    img = cv2.imread(str(img_path))
    img_shape = img.shape
    return img_shape[:2]


points_img_coords = calibration_utils.convert_to_img_coords(points, calib)
img_path = img_dir / format_filename(sample_id, "png")
img_shape = get_image_shape(img_path)
mask_fov = get_fov_mask(points_img_coords, img_shape)
points = points[mask_fov]
print("Points shape:", points.shape)
print('Min/Max values for all dimensions')
min_x, max_x, min_y, max_y, min_z, max_z = get_min_and_max_values(points)
print(f"X: ({min_x:.2f}, {max_x:.2f})")
print(f"Y: ({min_y:.2f}, {max_y:.2f})")
print(f"Z: ({min_z:.2f}, {max_z:.2f})")

Points shape: (20285, 3)
Min/Max values for all dimensions
X: (4.53, 73.04)
Y: (-16.13, 23.59)
Z: (-2.35, 2.64)


#### Mask Points and Boxes Outside Range

In [9]:
POINT_CLOUD_X_RANGE = 0, 70.4
POINT_CLOUD_Y_RANGE = -40, 40
POINT_CLOUD_Z_RANGE = -3, 1

def mask_points_outside_range(points):
    min_x, max_x = POINT_CLOUD_X_RANGE
    min_y, max_y = POINT_CLOUD_Y_RANGE
    mask = (points[:, 0] >= min_x) & (points[:, 0] <= max_x) \
           & (points[:, 1] >= min_y) & (points[:, 1] <= max_y)
    return points[mask]


points = mask_points_outside_range(points)
print("Points shape:", points.shape)
print('Min/Max values for all dimensions')
min_x, max_x, min_y, max_y, min_z, max_z = get_min_and_max_values(points)
print(f"X: ({min_x:.2f}, {max_x:.2f})")
print(f"Y: ({min_y:.2f}, {max_y:.2f})")
print(f"Z: ({min_z:.2f}, {max_z:.2f})")

Points shape: (20266, 3)
Min/Max values for all dimensions
X: (4.53, 59.86)
Y: (-16.13, 23.59)
Z: (-2.35, 1.94)


In [10]:
def mask_boxes_outside_range(gt_boxes):
    min_xyz = np.array([POINT_CLOUD_X_RANGE[0], POINT_CLOUD_Y_RANGE[0], POINT_CLOUD_Z_RANGE[0]])
    max_xyz = np.array([POINT_CLOUD_X_RANGE[1], POINT_CLOUD_Y_RANGE[1], POINT_CLOUD_Z_RANGE[1]])
    return [box for box in gt_boxes if np.all(box.center >= min_xyz) and np.all(box.center <= max_xyz)]


pcd = create_point_cloud(points)
gt_boxes = mask_boxes_outside_range(gt_boxes)
o3d.visualization.draw_geometries([pcd, *gt_boxes])

#### Sample Points

In [11]:
MAX_POINTS = 16384

def exceeds_max_points(points):
    return points.shape[0] > MAX_POINTS

def get_mask_points_near(points):
    points_dist = np.linalg.norm(points, axis=1)
    mask_points_near = points_dist < 40.0
    return mask_points_near

def get_far_and_near_idx(points):
    mask_points_near = get_mask_points_near(points)
    far_idx = np.where(mask_points_near == 0)[0]
    near_idx = np.where(mask_points_near == 1)[0]
    return far_idx, near_idx

def is_far_sparse(far_points):
    return far_points < MAX_POINTS

def retain_far_and_sample_near(points):
    far_idx, near_idx = get_far_and_near_idx(points)
    num_sample = max(0, MAX_POINTS - len(far_idx))
    sampled_near = np.random.choice(near_idx, num_sample, replace=False)
    choice = np.concatenate((far_idx, sampled_near)) if len(far_idx) > 0 else sampled_near
    np.random.shuffle(choice)
    return points[choice]

def sample_random(points):
    choice = np.random.choice(len(points), MAX_POINTS, replace=False)
    np.random.shuffle(choice)
    return points[choice]

def sample_distance_or_random(points):
    far_idx, near_idx = get_far_and_near_idx(points)
    if is_far_sparse(len(far_idx)):
        sampled_points = retain_far_and_sample_near(points)
    else:
        sampled_points = sample_random(points)
    return sampled_points

def sample_first(points):
    choice = np.random.choice(len(points), MAX_POINTS, replace=False)
    np.random.shuffle(choice)
    return points[choice]

def sample_points(points):
    if exceeds_max_points(points):
        sampled_points = sample_distance_or_random(points)
    else:
        sampled_points = sample_first(points)
    return sampled_points

points = sample_points(points)
print("Points shape:", points.shape)
print('Min/Max values for all dimensions')
min_x, max_x, min_y, max_y, min_z, max_z = get_min_and_max_values(points)
print(f"X: ({min_x:.2f}, {max_x:.2f})")
print(f"Y: ({min_y:.2f}, {max_y:.2f})")
print(f"Z: ({min_z:.2f}, {max_z:.2f})")

Points shape: (16384, 3)
Min/Max values for all dimensions
X: (4.53, 59.86)
Y: (-16.13, 23.59)
Z: (-2.35, 1.94)


In [12]:
pcd = create_point_cloud(points)
o3d.visualization.draw_geometries([pcd, *gt_boxes])